In [ ]:
!pip install ultralytics

In [ ]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
# Load YOLOv5 models

# from utils.general import non_max_suppression, scale_coords
# from utils.torch_utils import select_device, time_sync

# Define paths to your models


def adjust(segImg):
    # Adjust the segmentation image as needed
    im_width = segImg.width
    im_height = segImg.height
    upper_px = ()
    lower_px = ()
    
    # Example adjustment logic (modify as per your specific needs)
    for i in range(im_height):
        flag = 0
        for j in range(im_width):
            coordinate = j, i
            if segImg.getpixel(coordinate) == 1:
                upper_px = (j, i)
                flag = 1
                break
        
        if flag == 1:
            break
    
    for i in reversed(range(im_height)):
        flag = 0
        for j in range(im_width):
            coordinate = j, i
            if segImg.getpixel(coordinate) == 1:
                lower_px = (j, i)
                flag = 1
                break
        
        if flag == 1:
            break
    
    return upper_px, lower_px

def preprocess_image(image_path):
    # Read and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def segment_image(image):
    # Perform segmentation using YOLOv5 model (example, adjust as needed)
    imgsz = 640  # size of the input image
#     stride = int(seg_model.stride.max())  # model stride
    img = image.copy()

    # Inference
    img = torch.from_numpy(img)
    img = img.float() / 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = seg_model(img, augment=False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

    # Process results
    for i, det in enumerate(pred):  # detections per image
        if det is not None and len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image.shape).round()
    
    return pred

def detect_keypoints_side(image):
    # Perform side keypoint detection using YOLOv5 model (example, adjust as needed)
    imgsz = 640  # size of the input image
    stride = int(pose_side_model.stride.max())  # model stride
    img = image.copy()

    # Inference
    img = torch.from_numpy(img)
    img = img.float() / 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = pose_side_model(img, augment=False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

    # Process results
    for i, det in enumerate(pred):  # detections per image
        if det is not None and len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image.shape).round()

    return pred

def detect_keypoints_rear(image):
    # Perform rear keypoint detection using YOLOv5 model (example, adjust as needed)
    imgsz = 640  # size of the input image
    stride = int(pose_rear_model.stride.max())  # model stride
    img = image.copy()

    # Inference
    img = torch.from_numpy(img)
    img = img.float() / 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = pose_rear_model(img, augment=False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

    # Process results
    for i, det in enumerate(pred):  # detections per image
        if det is not None and len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image.shape).round()

    return pred

def draw_masks(image, masks):
    # Draw segmentation masks on the image
    # Example drawing logic (modify as per your specific needs)
    for mask in masks:
        mask = (mask * 255).astype(np.uint8)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
    return image

def draw_keypoints(image, keypoints, color=(0, 0, 255)):
    # Draw keypoints on the image
    # Example drawing logic (modify as per your specific needs)
    for kp_set in keypoints:
        for kp in kp_set:
            x, y = kp[:2]
            cv2.circle(image, (int(x), int(y)), 5, color, -1)
    return image

def infer(image_path):
    # Load and preprocess the image
    image = preprocess_image(image_path)

    # Perform segmentation
    masks = segment_image(image)
    image_with_masks = draw_masks(image.copy(), masks)

    # Perform side keypoint detection
    side_keypoints = detect_keypoints_side(image)
    image_with_side_keypoints = draw_keypoints(image_with_masks.copy(), side_keypoints, color=(255, 0, 0))

    # Perform rear keypoint detection
    rear_keypoints = detect_keypoints_rear(image)
    final_image = draw_keypoints(image_with_side_keypoints.copy(), rear_keypoints, color=(0, 0, 255))

    return final_image

# Example usage
image_path = '/kaggle/input/cow-side-back/1.0_r_117_10.0_F.jpg'
output_image = infer(image_path)
cv2.imshow('Output Image', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def inference_yolov8_pose(pose_model, img):
    # Perform pose estimation using the pose model
    results = pose_model.predict(img)
    keypoints = []
    for result in results:
        keypoints.append({
            "keypoints": np.array(result["keypoints"])
        })
    return keypoints

def inference_yolov8_seg(seg_model, img):
    # Perform segmentation using the segmentation model
    results = seg_model.predict(img)
    segmentation_masks = []
    for result in results:
        segmentation_masks.append(np.array(result["mask"]))
    return segmentation_masks

In [ ]:
import numpy as np
import joblib
from PIL import Image
import torch
from ultralytics import YOLO

# Assuming YOLOv8 models have similar init and inference functions

def adjust(segImg):
    im_width, im_height = segImg.size
    upper_px = lower_px = None

    for i in range(im_height):
        for j in range(im_width):
            if segImg.getpixel((j, i)) == 1:
                upper_px = (j, i)
                break
        if upper_px:
            break

    for i in reversed(range(im_height)):
        for j in range(im_width):
            if segImg.getpixel((j, i)) == 1:
                lower_px = (j, i)
                break
        if lower_px:
            break

    return upper_px, lower_px

def y_distance(point1, point2):
    return round(abs(point1[1] - point2[1]))

def load_models():
    path_to_seg_yolov8 = '/kaggle/input/cattle-4-models/Final 4 model weights/best_seg_yolov8l.pt'
    path_to_pose_side_yolo = '/kaggle/input/cattle-4-models/Final 4 model weights/best_pose_side_yolov8l.pt'
    path_to_pose_rear_yolo = '/kaggle/input/cattle-4-models/Final 4 model weights/best_pose_rear_yolov8l.pt'
    weight_filename = '/kaggle/input/cattle-4-models/Final 4 model weights/model_v3.joblib'

    seg_model = YOLO(path_to_seg_yolov8)
    pose_side_model = YOLO(path_to_pose_side_yolo)
    pose_rear_model = YOLO(path_to_pose_rear_yolo)

    weight_model = joblib.load(weight_filename)

    return seg_model, pose_rear_model, pose_side_model, weight_model

def process_image(det_model, seg_model, pose_model, img):
    # Perform segmentation inference
    seg_results = inference_yolov8_seg(seg_model, img)
    
    # Perform pose estimation inference
    pose_results = inference_yolov8_pose(pose_model, img)
    
    return pose_results, seg_results

def calculate_features(rear_pose_results, side_pose_results, side_seg_result, rear_seg_result):
    rear_kpt = rear_pose_results[0]["keypoints"][:, :2]
    side_kpt = side_pose_results[0]["keypoints"][:, :2]

    if side_kpt.shape != (9, 2) or rear_kpt.shape != (4, 2):
        raise ValueError("Incorrect keypoint shape")

    segImg = Image.fromarray(np.array(side_seg_result[0].astype('uint8')))
    segRear = Image.fromarray(np.array(rear_seg_result[0].astype('uint8')))
    rear_p1, rear_p2 = adjust(segRear)
    rear_height = y_distance(rear_p1, rear_p2)

    side_im_width, side_im_height = segImg.size
    if int(side_kpt[1, 0]) < (side_im_width / 2):
        seg_crop = segImg.crop((0, 0, int(side_kpt[8, 0]), side_im_height))
    else:
        seg_crop = segImg.crop((int(side_kpt[8, 0]), 0, side_im_width, side_im_height))

    side_p1, side_p2 = adjust(seg_crop)
    side_height = y_distance(side_p1, side_p2)
    side_Length_shoulderbone = round(np.linalg.norm(side_kpt[2] - side_kpt[1]))
    side_F_Girth = round(np.linalg.norm(side_kpt[4] - side_kpt[3]))
    side_R_Girth = round(np.linalg.norm(side_kpt[8] - side_kpt[7]))
    rear_width = round(np.linalg.norm(rear_kpt[1] - rear_kpt[0]))
    actual_width = rear_width * (side_height / rear_height)

    return side_Length_shoulderbone, side_F_Girth, side_R_Girth, actual_width

def predict_weight(weight_model, features, sticker, cattle):
    predicted_cattle_weight = float(weight_model.predict([features]))
    ratio = cattle / sticker

    adjustment_factors = [
        (50, 0.68), (55, 0.57), (60, 0.48), (65, 0.40), (67, 0.28),
        (70, 0), (72, 0.25), (75, 0.35), (80, 0.45), (85, 0.55),
        (90, 0.65), (95, 0.75), (100, 0.85), (105, 0.95), (110, 1.05),
        (115, 1.15), (120, 1.25), (125, 1.35), (130, 1.45), (135, 1.55),
        (140, 1.65), (145, 1.75), (150, 1.95), (155, 2.05), (160, 2.20),
        (165, 2.35), (170, 2.45), (180, 2.55), (190, 2.65), (200, 2.75),
        (210, 2.85), (220, 2.95), (230, 3.05)
    ]

    for threshold, factor in adjustment_factors:
        if ratio < threshold:
            predicted_cattle_weight -= ratio * factor
            break
    else:
        predicted_cattle_weight = 0
        status = "Sorry! This cattle can't be handled right now."
        return {"weight": predicted_cattle_weight, "ratio": ratio, "remarks": status}

    return {"weight": predicted_cattle_weight, "ratio": ratio, "remarks": "ok"}

def predict(side_fname, rear_fname):
    print(torch.__version__, torch.cuda.is_available())

    try:
        seg_model, rear_pose_model, side_pose_model, weight_model = load_models()
        rear_pose_results, rear_seg_result = process_image(None, seg_model, rear_pose_model, rear_fname)
        side_pose_results, side_seg_result = process_image(None, seg_model, side_pose_model, side_fname)

        seg = np.asarray(side_seg_result)
        sticker = (seg == 2).sum()
        cattle = (seg == 1).sum()

        if sticker < 100:
            return {"weight": 0, "ratio": cattle / sticker, "remarks": "Please apply sticker correctly."}

        features = calculate_features(rear_pose_results, side_pose_results, side_seg_result, rear_seg_result)
        return predict_weight(weight_model, features, sticker, cattle)

    except Exception as e:
        print(f"Error: {e}")
        return {"weight": 0, "ratio": 0, "remarks": "An error occurred during processing."}


In [ ]:
predict('/kaggle/input/cow-side-back/10.0_s_141_2.0_M.jpg','/kaggle/input/cow-side-back/10.0_r_141_2.0_M.jpg')

In [ ]:

def inference_yolov8_pose(pose_model, img):
    results = pose_model.predict(img)
    keypoints = []
    for result in results:
        keypoints.append({
            "keypoints": np.array(result.keypoints.cpu())  # Ensure keypoints are on CPU
        })
    return keypoints

def inference_yolov8_seg(seg_model, img):
    results = seg_model.predict(img)
    segmentation_masks = []
    for result in results:
        segmentation_masks.append(np.array(result.masks.cpu()))  # Ensure masks are on CPU
    return segmentation_masks

def process_image(pose_model, seg_model, img):
    seg_results = inference_yolov8_seg(seg_model, img)
    pose_results = inference_yolov8_pose(pose_model, img)
    return pose_results, seg_results

def predict(side_fname, rear_fname):
    print(torch.__version__, torch.cuda.is_available())

    try:
        seg_model, rear_pose_model, side_pose_model, weight_model = load_models()
        rear_pose_results, rear_seg_result = process_image(rear_pose_model, seg_model, rear_fname)
        side_pose_results, side_seg_result = process_image(side_pose_model, seg_model, side_fname)

        # Debugging: print shapes and types of results
        print(f"rear_pose_results: {rear_pose_results}")
        print(f"rear_seg_result: {rear_seg_result}")
        print(f"side_pose_results: {side_pose_results}")
        print(f"side_seg_result: {side_seg_result}")

        seg = np.asarray(side_seg_result[0])
        print(f"seg shape: {seg.shape}, seg dtype: {seg.dtype}")
        
        sticker = (seg == 2).sum()
        cattle = (seg == 1).sum()

        print(f"sticker count: {sticker}, cattle count: {cattle}")

        if sticker < 100:
            return {"weight": 0, "ratio": cattle / sticker, "remarks": "Please apply sticker correctly."}

        features = calculate_features(rear_pose_results, side_pose_results, side_seg_result, rear_seg_result)
        return predict_weight(weight_model, features, sticker, cattle)

    except Exception as e:
        print(f"Error: {e}")
        return {"weight": 0, "ratio": 0, "remarks": "An error occurred during processing."}


In [ ]:
predict('/kaggle/input/cow-side-back/10.0_s_141_2.0_M.jpg','/kaggle/input/cow-side-back/10.0_r_141_2.0_M.jpg')

In [ ]:
def inference_yolov8_pose(pose_model, img):
    # Perform pose estimation using the pose model
    results = pose_model.predict(img)
    keypoints = []
    for result in results:
        keypoints.append({
            "keypoints": np.array(result.keypoints.data.cpu(), dtype=np.float32)  # Ensure keypoints are on CPU and float32
        })
    return keypoints

def inference_yolov8_seg(seg_model, img):
    # Perform segmentation using the segmentation model
    results = seg_model.predict(img)
    segmentation_masks = []
    for result in results:
        segmentation_masks.append(np.array(result.masks.data.cpu(), dtype=np.uint8))  # Ensure masks are on CPU and uint8
    return segmentation_masks
def process_image(pose_model, seg_model, img):
    seg_results = inference_yolov8_seg(seg_model, img)
    pose_results = inference_yolov8_pose(pose_model, img)
    return pose_results, seg_results

def calculate_features(rear_pose_results, side_pose_results, side_seg_result, rear_seg_result, sticker, cattle):
    
    rear_kpt = rear_pose_results[0]["keypoints"][0][:, :2]
    side_kpt = side_pose_results[0]["keypoints"][0][:, :2]
    

    if side_kpt.shape != (9, 2) or rear_kpt.shape != (4, 2):
        raise ValueError("Incorrect keypoint shape")
        
    segImg = Image.fromarray(np.array(side_seg_result[0][0].astype('uint8')))
    segRear = Image.fromarray(np.array(rear_seg_result[0][0].astype('uint8')))
    rear_p1, rear_p2 = adjust(segRear)
    print(rear_p1, rear_p2)
    rear_height = y_distance(rear_p1, rear_p2)

    side_im_width, side_im_height = segImg.size
    if int(side_kpt[1, 0]) < (side_im_width / 2):
        seg_crop = segImg.crop((0, 0, int(side_kpt[8, 0]), side_im_height))
    else:
        seg_crop = segImg.crop((int(side_kpt[8, 0]), 0, side_im_width, side_im_height))

    side_p1, side_p2 = adjust(seg_crop)
    side_height = y_distance(side_p1, side_p2)
    side_Length_shoulderbone = round(np.linalg.norm(side_kpt[2] - side_kpt[1]))
    side_F_Girth = round(np.linalg.norm(side_kpt[4] - side_kpt[3]))
    side_R_Girth = round(np.linalg.norm(side_kpt[8] - side_kpt[7]))
    rear_width = round(np.linalg.norm(rear_kpt[1] - rear_kpt[0]))
    actual_width = rear_width * (side_height / rear_height)
    #side_Length_shoulderbone,side_F_Girth,	side_R_Girth, sticker, cattle , actual_width
    return [side_Length_shoulderbone, side_F_Girth, side_R_Girth, sticker, cattle, actual_width]

def predict_weight(weight_model, features, sticker, cattle):
    print('side_Length_shoulderbone,side_F_Girth,	side_R_Girth, sticker, cattle , actual_width')
    print(features)
    predicted_cattle_weight = float(weight_model.predict([features]))
    ratio = cattle / sticker

    adjustment_factors = [
        (50, 0.68), (55, 0.57), (60, 0.48), (65, 0.40), (67, 0.28),
        (70, 0), (72, 0.25), (75, 0.35), (80, 0.45), (85, 0.55),
        (90, 0.65), (95, 0.75), (100, 0.85), (105, 0.95), (110, 1.05),
        (115, 1.15), (120, 1.25), (125, 1.35), (130, 1.45), (135, 1.55),
        (140, 1.65), (145, 1.75), (150, 1.95), (155, 2.05), (160, 2.20),
        (165, 2.35), (170, 2.45), (180, 2.55), (190, 2.65), (200, 2.75),
        (210, 2.85), (220, 2.95), (230, 3.05)
    ]

    for threshold, factor in adjustment_factors:
        if ratio < threshold:
            predicted_cattle_weight += ratio * factor
            break
    else:
        predicted_cattle_weight = 0
        status = "Sorry! This cattle can't be handled right now."
        return {"weight": predicted_cattle_weight, "ratio": ratio, "remarks": status}

    return {"weight": predicted_cattle_weight, "ratio": ratio, "remarks": "ok"}

def predict(side_fname, rear_fname):
    print(torch.__version__, torch.cuda.is_available())

    try:
        seg_model, rear_pose_model, side_pose_model, weight_model = load_models()
        rear_pose_results, rear_seg_result = process_image(rear_pose_model, seg_model, rear_fname)
        side_pose_results, side_seg_result = process_image(side_pose_model, seg_model, side_fname)

        # Debugging: print shapes and types of results
#         print(f"rear_pose_results: {rear_pose_results}")
#         print(f"rear_seg_result: {rear_seg_result}")
#         print(f"side_pose_results: {side_pose_results}")
#         print(f"side_seg_result: {side_seg_result}")
#         print(side_seg_result[0])
        for i in range(len(side_seg_result[0][1])):
            for j in range(len(side_seg_result[0][1][i])):
                if(side_seg_result[0][1][i][j]==1): 
                    side_seg_result[0][1][i][j]=2
        seg = np.asarray(side_seg_result[0])
        print(f"seg shape: {seg.shape}, seg dtype: {seg.dtype}")
        
        sticker = (seg == 2).sum()
        cattle = (seg == 1).sum()

        print(f"sticker count: {sticker}, cattle count: {cattle}")

        if sticker < 100:
            return {"weight": 0, "ratio": cattle / sticker, "remarks": "Please apply sticker correctly."}
        features = calculate_features(rear_pose_results, side_pose_results, side_seg_result, rear_seg_result, sticker, cattle)
        return predict_weight(weight_model, features, sticker, cattle)

    except Exception as e:
        print(f"Error: {e}")
        return {"weight": 0, "ratio": 0, "remarks": "An error occurred during processing."}

In [ ]:
predict('/kaggle/input/cow-side-back/1.0_s_117_10.0_F.jpg','/kaggle/input/cow-side-back/1.0_r_117_10.0_F.jpg')

In [ ]:
weight_filename = '/kaggle/input/cattle-4-models/Final 4 model weights/model_v3.joblib'
weight_model = joblib.load(weight_filename)

In [ ]:
float(weight_model.predict([[467, 477, 436, 2166, 106132, 355.49612403100775]]))

In [ ]:
path_to_pose_side_yolo = '/kaggle/input/cattle-4-models/Final 4 model weights/best_pose_side_yolov8l.pt'
side_pose_model = YOLO(path_to_pose_side_yolo)
def inference_yolov8_pose(pose_model, img):
    # Perform pose estimation using the pose model
    results = pose_model.predict(img)
    keypoints = []
    for result in results:
        keypoints.append({
            "keypoints": np.array(result.keypoints.data.cpu(), dtype=np.float32)  # Ensure keypoints are on CPU and float32
        })
    return keypoints

np.asarray(inference_yolov8_pose(side_pose_model,'/kaggle/input/cow-side-back/1.0_s_117_10.0_F.jpg')[0])

In [ ]:
ab = [np.array([[0, 0, 1], [0, 1, 1], [1, 1, 1]])]

In [ ]:
seg = np.asarray(ab[0])

In [ ]:
(seg==1).sum()

In [ ]:
ab = ab.reshape(-1)

In [ ]:
st = set()

In [ ]:
for i in ab:
    st.add(i)

In [ ]:
st

In [ ]:
path_to_pose_rear_yolo = '/kaggle/input/cattle-4-models/Final 4 model weights/best_seg_yolov8l.pt'
seg_model = YOLO(path_to_pose_rear_yolo)
def inference_yolov8_seg(seg_model, img):
    # Perform segmentation using the segmentation model
    results = seg_model.predict(img)
    segmentation_masks = []
    for result in results:
        segmentation_masks.append(np.array(result.masks.data.cpu(), dtype=np.uint8))  # Ensure masks are on CPU and uint8
    return segmentation_masks
inference_yolov8_seg(seg_model,'/kaggle/input/cow-side-back/1.0_s_117_10.0_F.jpg')

In [ ]:
ab = seg_model.predict('/kaggle/input/cow-side-back/1.0_s_117_10.0_F.jpg')[0].masks.data.numpy()

In [ ]:
ab[0][0]

In [ ]:
(ab[0][1].reshape(-1)==1).sum()

In [ ]:
for i in range(len(ab[1])):
    for j in range(len(ab[1][i])):
        if(ab[1][i][j]==1): 
            ab[1][i][j]=2

In [ ]:
(ab[1].reshape(-1)==2).sum()

In [ ]:
st = set()

In [ ]:
for i in ab:
    st.add(i)

In [ ]:
st

In [ ]:
a = np.array([1,2,3,4])